In [1]:
import os
os.chdir('..')

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
from three_tank_data.data_gen import ThreeTankDataGenerator
import pandas as pd
import constants as const
import pysindy as ps
import numpy as np
import plotly.express as px
from ipywidgets import interact
import ipywidgets as widgets
from plotly.subplots import make_subplots
import plotly.graph_objects as go
%load_ext autoreload
%autoreload 2

In [4]:
df_z = pd.read_parquet(const.Z_SPACE_DATA_PATH)

In [5]:
df_z.head()

,h1,h2,h3,h1_dot,h2_dot,h3_dot,time,uid_initial_state
0,64.602324,27.307063,17.744896,-2.697734,1.328168,1.369567,0.0,0
1,64.332551,27.439879,17.881853,-2.683108,1.313889,1.369219,0.1,0
2,64.064240,27.571268,18.018775,-2.668506,1.299728,1.368778,0.2,0
3,63.797389,27.701241,18.155653,-2.653931,1.285700,1.368231,0.3,0
4,63.531996,27.829811,18.292476,-2.639382,1.271790,1.367592,0.4,0


In [6]:
def _create_ts_plot(idx):
    df_plot = df_z[df_z[const.UID_INITIAL_STATE_COL_NAME] == idx]
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    # signal 1
    for col, name in zip(const.Z_COL_NAMES, ['h1(t)', 'h2(t)', 'h3(t)']):
        fig.add_trace(go.Scatter(x=df_plot.time, y=df_plot[col], name=name,
                      mode="lines", opacity=1),
            row=1, col=1)

    for col, name in zip(const.Z_DOT_COL_NAMES, ['h1_dot(t)', 'h2_dot(t)', 'h3_dot(t)']):
        fig.add_trace(go.Scatter(x=df_plot.time, y=df_plot[col], name=name,
                      mode="lines", opacity=1),
            row=2, col=1)

    fig.update_xaxes(title_text='time')
    fig.update_yaxes(title_text='x', row=1)
    fig.update_yaxes(title_text='x_dot', row=2)
    fig.update_layout(title_text="Latent neuron activations vs. hidden states", showlegend=True)
    fig.show()

interact(_create_ts_plot, idx=list(range(const.NUMBER_INITIAL_STATES)))

interactive(children=(Dropdown(description='idx', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1…

<function __main__._create_ts_plot(idx)>

# Lets try the pysindy exercise in the z-space!

In [7]:
library_functions = [
    lambda x : np.exp(x),
    lambda x : 1./x,
    lambda x : x,
    lambda x : np.sin(x),
    lambda x : np.cos(x),
    lambda x,y : np.sin(x+y),
    lambda x,y : np.cos(x+y),
    lambda x,y : np.sign(x-y)*np.sqrt(np.abs(x-y)),
    lambda x: x**2,
    lambda x: np.sqrt(x),
    lambda x,y: x*y,
    lambda x,y: x**2*y,
    lambda x,y: x*y**2
]
library_function_names = [
    lambda x : 'exp(' + x + ')',
    lambda x : '1/' + x,
    lambda x : x,
    lambda x : f'sin({x})',
    lambda x : f'cos({x})',
    lambda x,y : 'sin(' + x + '+' + y + ')',
    lambda x,y : f'cos({x}+{y})',
    lambda x,y : 'sign('+x+'-'+y+')*sqrt('+x+' - '+y+')',
    lambda x: x+'^2',
    lambda x: f'sqrt({x})',
    lambda x,y: f'{x}*{y}',
    lambda x,y: f'{x}^2*{y}',
    lambda x,y: f'{x}*{y}^2'
]
feature_library = ps.CustomLibrary(
    library_functions=library_functions, function_names=library_function_names
)

In [8]:
optimizer = ps.STLSQ(threshold=.1)
model = ps.SINDy(
    feature_library=feature_library,
    optimizer=optimizer,
    feature_names=["h1", "h2", "h3"]
)
model.fit(df_z[const.Z_COL_NAMES].values, x_dot=df_z[const.Z_DOT_COL_NAMES].values, t=0.1)
model.print()

h1' = -0.441 sign(h1-h2)*sqrt(h1 - h2)
h2' = 0.439 sign(h1-h2)*sqrt(h1 - h2) + -0.439 sign(h2-h3)*sqrt(h2 - h3)
h3' = 0.441 sign(h2-h3)*sqrt(h2 - h3)


/home/henrik/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning:

Ill-conditioned matrix (rcond=3.49556e-81): result may not be accurate.

/home/henrik/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning:

Ill-conditioned matrix (rcond=3.49556e-81): result may not be accurate.

/home/henrik/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning:

Ill-conditioned matrix (rcond=3.49556e-81): result may not be accurate.



## NICE!

In [9]:
feature_library.get_feature_names()

['exp(x0)',
 'exp(x1)',
 'exp(x2)',
 '1/x0',
 '1/x1',
 '1/x2',
 'x0',
 'x1',
 'x2',
 'sin(x0)',
 'sin(x1)',
 'sin(x2)',
 'cos(x0)',
 'cos(x1)',
 'cos(x2)',
 'sin(x0+x1)',
 'sin(x0+x2)',
 'sin(x1+x2)',
 'cos(x0+x1)',
 'cos(x0+x2)',
 'cos(x1+x2)',
 'sign(x0-x1)*sqrt(x0 - x1)',
 'sign(x0-x2)*sqrt(x0 - x2)',
 'sign(x1-x2)*sqrt(x1 - x2)',
 'x0^2',
 'x1^2',
 'x2^2',
 'sqrt(x0)',
 'sqrt(x1)',
 'sqrt(x2)',
 'x0*x1',
 'x0*x2',
 'x1*x2',
 'x0^2*x1',
 'x0^2*x2',
 'x1^2*x2',
 'x0*x1^2',
 'x0*x2^2',
 'x1*x2^2']

# Now in the x space!

In [10]:
df_x = pd.read_parquet(const.X_SPACE_DATA_PATH)

def plot_pics(sample_index):
    df_plot_x = df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == sample_index].copy().reset_index(drop=True)
    sub_indexes = [i for i in range(5)]
    img_sequence = [df_plot_x[const.X_COL_NAMES].values[i,:].reshape(
        (const.PICTURE_SIZE, const.PICTURE_SIZE)) for i in sub_indexes]
    fig = px.imshow(np.array(img_sequence),
                    facet_col=0,
#                     binary_string=True,
                    labels={'facet_col':'sigma'}
                   )
    for i, index in enumerate(sub_indexes):
        fig.layout.annotations[i]['text'] = f'Timestamp {index}'
    fig.show()

interact(plot_pics, sample_index=range(const.NUMBER_INITIAL_STATES))

interactive(children=(Dropdown(description='sample_index', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, …

<function __main__.plot_pics(sample_index)>

In [11]:
[i for i in range(10,15)]

[10, 11, 12, 13, 14]

In [12]:
def plot_pics(sample_index):
    df_plot_x = df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == sample_index].copy().reset_index(drop=True)
    sub_indexes = [i for i in range(20,30)]
    img_sequence = [df_plot_x[const.XDOT_COL_NAMES].values[i,:].reshape(
        (const.PICTURE_SIZE, const.PICTURE_SIZE)) for i in sub_indexes]
    fig = px.imshow(np.array(img_sequence),
                    facet_col=0,
#                     binary_string=True,
                    labels={'facet_col':'sigma'}
                   )
    for i, index in enumerate(sub_indexes):
        fig.layout.annotations[i]['text'] = f'Timestamp {index}'
    fig.show()

interact(plot_pics, sample_index=range(const.NUMBER_INITIAL_STATES))

interactive(children=(Dropdown(description='sample_index', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, …

<function __main__.plot_pics(sample_index)>

In [13]:
df_x = pd.read_parquet(const.X_SPACE_DATA_PATH)

def plot_pics(sample_index):
    df_plot_x = df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == sample_index].copy().reset_index(drop=True)
    sub_indexes = [0, 25, 50, 75, 100]
    img_sequence = [df_plot_x[const.X_COL_NAMES].values[i,:].reshape(
        (const.PICTURE_SIZE, const.PICTURE_SIZE)) for i in sub_indexes]
    fig = px.imshow(np.array(img_sequence),
                    facet_col=0,
#                     binary_string=True,
                    labels={'facet_col':'sigma'}
                   )
    for i, index in enumerate(sub_indexes):
        fig.layout.annotations[i]['text'] = f'Timestamp {index}'
    fig.show()

interact(plot_pics, sample_index=range(const.NUMBER_INITIAL_STATES))

interactive(children=(Dropdown(description='sample_index', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, …

<function __main__.plot_pics(sample_index)>

## What about the derivatives in x?

In [14]:
def plot_pics(sample_index):
    df_plot_x = df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == sample_index].copy().reset_index(drop=True)
    sub_indexes = [0, 25, 50, 75, 100]
    img_sequence = [df_plot_x[const.XDOT_COL_NAMES].values[i,:].reshape(
        (const.PICTURE_SIZE, const.PICTURE_SIZE)) for i in sub_indexes]
    fig = px.imshow(np.array(img_sequence),
                    facet_col=0,
#                     binary_string=True,
                    labels={'facet_col':'sigma'}
                   )
    for i, index in enumerate(sub_indexes):
        fig.layout.annotations[i]['text'] = f'Timestamp {index}'
    fig.show()

interact(plot_pics, sample_index=range(const.NUMBER_INITIAL_STATES))

interactive(children=(Dropdown(description='sample_index', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, …

<function __main__.plot_pics(sample_index)>

In [15]:
x_dot = df_x[const.XDOT_COL_NAMES].values
x = df_x[const.X_COL_NAMES].values

In [16]:
i = 10

In [17]:
px.imshow(x[i,:].reshape(100, 100))

In [18]:
px.imshow(x_dot[i,:].reshape(100, 100))

In [19]:
df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == 0]

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,xdot_9995,xdot_9996,xdot_9997,xdot_9998,xdot_9999,h1,h2,h3,time,uid_initial_state
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.602324,27.307063,17.744896,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.332551,27.439879,17.881853,0.1,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.064240,27.571268,18.018775,0.2,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,63.797389,27.701241,18.155653,0.3,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,63.531996,27.829811,18.292476,0.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,45.034976,34.963893,29.655414,9.6,0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,44.895032,35.002088,29.757163,9.7,0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,44.756336,35.039649,29.858298,9.8,0
99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,44.618886,35.076584,29.958813,9.9,0


In [20]:
df_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12928 entries, 0 to 12927
Columns: 20005 entries, x_0 to uid_initial_state
dtypes: float64(20004), int64(1)
memory usage: 1.9 GB


In [21]:
pd.Series([df_x[col].max() for col in const.X_COL_NAMES]).value_counts()

1.000000    8640
0.000000    1264
0.709045      32
0.943075      32
0.939516      32
dtype: int64

In [22]:
df_test = df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == 0]

In [23]:
df_test[const.X_COL_NAMES].std()[df_test[const.X_COL_NAMES].std()>0.069]

x_3500    0.194447
x_3501    0.194447
x_3502    0.194447
x_3503    0.194447
x_3504    0.194447
            ...   
x_8095    0.210869
x_8096    0.210869
x_8097    0.210869
x_8098    0.210869
x_8099    0.210869
Length: 1248, dtype: float64

In [24]:
x = np.array([[1, 2, 3],[4,5,6],[7,8,9]])
x

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [25]:
x.ravel()

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [26]:
33*100+1

3301

In [27]:
set([df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == 0][f'x_{i}'].max() for i in range(1000)])

{0.0}

In [28]:
def plot_stuff(i=3504):
    fig = px.scatter(df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == 0][f'x_{i}'])
#     fig = px.scatter(df_x[f'x_{i}'])

    fig.show()
interact(plot_stuff, i = [i for i in range(10000)])
    

interactive(children=(Dropdown(description='i', index=3504, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,…

<function __main__.plot_stuff(i=3504)>

In [29]:
def plot_stuff(i=3504):
    fig = px.scatter(df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == 0][f'xdot_{i}'])
#     fig = px.scatter(df_x[f'x_{i}'])

    fig.show()
interact(plot_stuff, i = [i for i in range(10000)])

interactive(children=(Dropdown(description='i', index=3504, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,…

<function __main__.plot_stuff(i=3504)>

In [30]:
from three_tank_data.data_module import ThreeTankDataModule
dm = ThreeTankDataModule(.1, 10, 10)